# Dataset

In [1]:
#First you have to build your vocab
#for example I am going to take only five sentence as dataset




dataset = ['I like going to the cinema.',
           'Mum is going to cook spinach tonight.',
           'Do you think they are going to win the match ?',
           'Are you going to the United States ?',
           "He doesn't like playing video games."
          ]


# unique_words

In [2]:
#build vocab 

#you can also remove stopwords if you want

vocab=[]

symbols = {0: 'PAD',1: 'UNK'}

for sentence in dataset:
    for word in sentence.split():
        vocab.append(word.lower())

vocab = list(set(vocab))



# Load glove or any word_embedding

In [10]:
#first method

import numpy as np
#load glove or any word embedding

def word_embedding_matrix(embedding_path,vocab,dim):
    
    #first and second vector are pad and unk words
    
    with open(embedding_path,'r') as f:
        word_vocab =[]
        embedding_matrix = []
        word_vocab.extend(['PAD','UNK'])
        embedding_matrix.append(np.random.uniform(-1.0, 1.0, (1,100))[0])
        embedding_matrix.append(np.random.uniform(-1.0, 1.0, (1,100))[0])

        
        for line in f:
            if line.split()[0] in vocab:
                word_vocab.append(line.split()[0])
                embedding_matrix.append([float(i) for i in line.split()[1:]])
                
        
    return {'word_vocab': word_vocab,'Embedding_matrix': np.reshape(embedding_matrix,[-1,100]).astype(np.float32)}

print(word_embedding_matrix('./glove.6B.100d.txt',vocab,100)['Embedding_matrix'][2])

    

[-0.038194 -0.24487   0.72812  -0.39961   0.083172  0.043953 -0.39141
  0.3344   -0.57545   0.087459  0.28787  -0.06731   0.30906  -0.26384
 -0.13231  -0.20757   0.33395  -0.33848  -0.31743  -0.48336   0.1464
 -0.37304   0.34577   0.052041  0.44946  -0.46971   0.02628  -0.54155
 -0.15518  -0.14107  -0.039722  0.28277   0.14393   0.23464  -0.31021
  0.086173  0.20397   0.52624   0.17164  -0.082378 -0.71787  -0.41531
  0.20335  -0.12763   0.41367   0.55187   0.57908  -0.33477  -0.36559
 -0.54857  -0.062892  0.26584   0.30205   0.99775  -0.80481  -3.0243
  0.01254  -0.36942   2.2167    0.72201  -0.24978   0.92136   0.034514
  0.46745   1.1079   -0.19358  -0.074575  0.23353  -0.052062 -0.22044
  0.057162 -0.15806  -0.30798  -0.41625   0.37972   0.15006  -0.53212
 -0.2055   -1.2526    0.071624  0.70565   0.49744  -0.42063   0.26148
 -1.538    -0.30223  -0.073438 -0.28312   0.37104  -0.25217   0.016215
 -0.017099 -0.38984   0.87424  -0.72569  -0.51058  -0.52028  -0.1459
  0.8278    0.27062 ]

# build int_to_vocab and vocab_to_int

In [11]:
matrix = word_embedding_matrix('./glove.6B.100d.txt',vocab,100)
load_embedding_matrix = matrix['Embedding_matrix']
shape_word_vocab      = matrix['word_vocab']

int_to_vocab = {}

for index_no,word in enumerate(shape_word_vocab):
    int_to_vocab[index_no] = word
int_to_vocab.update(symbols)


vocab_to_int = {word:index_no for index_no , word in int_to_vocab.items()}

print(len(shape_word_vocab))

print(vocab_to_int)
print(len(load_embedding_matrix))



24
{'PAD': 0, 'UNK': 1, 'the': 2, 'to': 3, 'is': 4, 'he': 5, 'are': 6, 'they': 7, 'i': 8, 'you': 9, 'do': 10, 'united': 11, 'states': 12, 'like': 13, '?': 14, 'going': 15, 'think': 16, 'win': 17, 'match': 18, 'playing': 19, 'video': 20, 'cook': 21, 'spinach': 22, 'mum': 23}
24


# Encode the sentences using vocab

In [12]:
#now encode the sentences using word_vocab

# word_vocab = word_embedding_matrix('./glove.6B.100d.txt',vocab,100)['word_vocab']
encoded_data = []

for sentence in dataset:
    sentence_ =[]
    for word in sentence.split():
        if word.lower() in vocab_to_int:
            sentence_.append(vocab_to_int[word.lower()])
    encoded_data.append(sentence_)
    
print(encoded_data)
    

#check
for i in encoded_data:
    print([int_to_vocab[j] for j in i ])

[[8, 13, 15, 3, 2], [23, 4, 15, 3, 21, 22], [10, 9, 16, 7, 6, 15, 3, 17, 2, 18, 14], [6, 9, 15, 3, 2, 11, 12, 14], [5, 13, 19, 20]]
['i', 'like', 'going', 'to', 'the']
['mum', 'is', 'going', 'to', 'cook', 'spinach']
['do', 'you', 'think', 'they', 'are', 'going', 'to', 'win', 'the', 'match', '?']
['are', 'you', 'going', 'to', 'the', 'united', 'states', '?']
['he', 'like', 'playing', 'video']


# Using Embedding matrix in Tensorflow

In [13]:
#Now let's use that matrix in tensorflow
import  tensorflow  as tf

tf.reset_default_graph()

sentences = tf.placeholder(tf.int32,
                           shape=[None,None]
                          )

Word_embedding = tf.get_variable(name="Word_embedding", 
                                 shape=[24,100], 
                                 initializer=tf.constant_initializer(np.array(load_embedding_matrix)), 
                                 trainable=False
                                )


embedding_loopup= tf.nn.embedding_lookup(Word_embedding,sentences)

with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for embedding_vector in encoded_data:
            print(sess.run(embedding_loopup,feed_dict={sentences:[embedding_vector]}))

[[[-4.6539e-02  6.1966e-01  5.6647e-01 -4.6584e-01 -1.1890e+00
    4.4599e-01  6.6035e-02  3.1910e-01  1.4679e-01 -2.2119e-01
    7.9239e-01  2.9905e-01  1.6073e-01  2.5324e-02  1.8678e-01
   -3.1001e-01 -2.8108e-01  6.0515e-01 -1.0654e+00  5.2476e-01
    6.4152e-02  1.0358e+00 -4.0779e-01 -3.8011e-01  3.0801e-01
    5.9964e-01 -2.6991e-01 -7.6035e-01  9.4222e-01 -4.6919e-01
   -1.8278e-01  9.0652e-01  7.9671e-01  2.4825e-01  2.5713e-01
    6.2320e-01 -4.4768e-01  6.5357e-01  7.6902e-01 -5.1229e-01
   -4.4333e-01 -2.1867e-01  3.8370e-01 -1.1483e+00 -9.4398e-01
   -1.5062e-01  3.0012e-01 -5.7806e-01  2.0175e-01 -1.6591e+00
   -7.9195e-02  2.6423e-02  2.2051e-01  9.9714e-01 -5.7539e-01
   -2.7266e+00  3.1448e-01  7.0522e-01  1.4381e+00  9.9126e-01
    1.3976e-01  1.3474e+00 -1.1753e+00  3.9503e-03  1.0298e+00
    6.4637e-02  9.0887e-01  8.2872e-01 -4.7003e-01 -1.0575e-01
    5.9160e-01 -4.2210e-01  5.7331e-01 -5.4114e-01  1.0768e-01
    3.9784e-01 -4.8744e-02  6.4596e-02 -6.1437e-01 -2.8

In [ ]:
#second method

def word_embedding_matrix(path,dim):
    word_id = {}
    with open(path) as fp:
        vocabulary = []
        embedding_matrix = [np.random.uniform(-1.0, 1.0, (dim, 1))]
        for line in fp.readlines():
            if line.split()[0] in vocab:
                vocabulary.append(line.split()[0])
                
                embedding_matrix.append(np.reshape(np.array(line.split()[1:]), (dim, 1)))
    word_id['PAD'] = 0
    word_id['UNK'] = len(vocabulary) + 1
    embedding_matrix.append(np.random.uniform(-1.0, 1.0, (dim, 1)))
    embedding_matrix = np.reshape(embedding_matrix, [-1, dim])
    embedding_matrix = embedding_matrix.astype(np.float32)

        
    return embedding_matrix , vocabulary

embedding_matrix,words2ids = word_embedding_matrix('./glove.6B.100d.txt',100)
print(embedding_matrix[1])